This Notebook handles the code for the classification model

In [63]:
%pip install torch torchvision torchaudio
%pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [64]:
%pip install molvs

Note: you may need to restart the kernel to use updated packages.


In [65]:
import os
import pickle
import torch
from torch import nn
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
import numpy as np
from rdkit import Chem
from torch_geometric.utils import from_smiles
import pandas as pd
from molvs import standardize_smiles
from sklearn.ensemble import RandomForestClassifier
CONFIG = {
    'data_dir': './processed_tox21',
    'hidden_channels': 128,
    'num_layers': 3,
    'dropout': 0.2,
    'batch_size': 64,
    'lr': 1e-3,
    'weight_decay': 0,
    'epochs': 50,
    'patience': 8,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Automatically detects if you have a GPU
print(f"Using device: {CONFIG['device']}")


Using device: cpu


Load data from preprocessing

In [66]:
def load_split(name):
    path = os.path.join(CONFIG['data_dir'], f'tox21_{name}.pkl')
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data


data_train = load_split('train')
data_validation = load_split('validation')
data_test = load_split('test')

print(f"Train: {len(data_train['smiles'])} | Validation: {len(data_validation['smiles'])} | Test: {len(data_test['smiles'])}")

Train: 6258 | Validation: 782 | Test: 783


Simplify the labels

In [67]:
data_train['labels'].shape

(6258, 12)

In [68]:
cols = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12']
y_train = pd.DataFrame(data_train['labels'], columns=cols).fillna(0)
y_test = pd.DataFrame(data_test['labels'], columns=cols).fillna(0)


# extract labels
y_train_label = pd.DataFrame(y_train.max(axis=1), columns=['toxic'])
y_test_label = pd.DataFrame(y_test.max(axis=1), columns=['toxic'])

In [69]:
smiles_train = data_train['smiles']
smiles_test = data_test['smiles']

Convert SMILES into ECFP

In [ ]:
# Code from https://drzinph.com/ECFP-fingerprints-in-python-part-3/
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs
class ECFP:
    def __init__(self, smiles):
        self.mols = [Chem.MolFromSmiles(i) for i in smiles]
        self.smiles = smiles

    def mol2fp(self, mol, radius = 3):
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius = radius)
        # fp = AllChem.MorganGenerator(mol, radius = radius)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array

    def compute_ECFP(self):
        bit_headers = ['bit' + str(i) for i in range(2048)]
        arr = np.empty((0,2048), int).astype(int)
        for i in self.mols:
            fp = self.mol2fp(i)
            arr = np.vstack((arr, fp))
        df_ECFP = pd.DataFrame(np.asarray(arr).astype(int),columns=bit_headers)
        df_ECFP.insert(loc=0, column='smiles', value=self.smiles)
        return df_ECFP

In [71]:
smiles = [standardize_smiles(i) for i in smiles_train] 
sm = [standardize_smiles(i) for i in smiles_test] 

[11:29:08] WARNING: not removing hydrogen atom without neighbors
[11:29:08] Can't kekulize mol.  Unkekulized atoms: 3 10


In [ ]:
# train set
ECFP_descriptor_train = ECFP(smiles)        # create your ECFP object and provide smiles
x_train_ecfp = ECFP_descriptor_train.compute_ECFP() # compute

# test set
ECFP_descriptor_test = ECFP(sm)        # create your ECFP object and provide smiles
x_test_ecfp = ECFP_descriptor_test.compute_ECFP() # compute

[11:29:10] WARNING: not removing hydrogen atom without neighbors
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganGenerator
[11:29:10] DEPRECATION WARNING: please use MorganG

In [73]:
train_data = pd.concat([x_train_ecfp, y_train_label], axis=1)
train_data.head()

,smiles,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,...,bit2039,bit2040,bit2041,bit2042,bit2043,bit2044,bit2045,bit2046,bit2047,toxic
0,COC(=O)[C@H]1CC[C@H](C(=O)OC)CC1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,O=S1(=O)CC(Cl)(Cl)C(Cl)(Cl)C1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,Cc1cc(C)c(NC(=O)C[C@H](CC(=O)[O-])c2cccc3ccccc...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1.0
3,Cc1noc(NS(=O)(=O)c2ccc(N)cc2)c1C,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
4,CC(C)(C)NC[C@H](O)COc1nsnc1N1CCOCC1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [74]:

x_train_ecfp.iloc[:, 1:]

,bit0,bit1,bit2,bit3,bit4,bit5,bit6,bit7,bit8,bit9,...,bit2038,bit2039,bit2040,bit2041,bit2042,bit2043,bit2044,bit2045,bit2046,bit2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6253,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6254,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Random Forest

In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42
)

In [125]:

rf.fit(x_train_ecfp.iloc[:, 1:], y_train_label)

/Users/vinceflores/anaconda3/lib/python3.13/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [126]:
y_pred = rf.predict(x_test_ecfp.iloc[:, 1:])

In [129]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
print("Accuracy:", accuracy_score(y_test_label, y_pred))
print("\nClassification Report:\n", classification_report(y_test_label, y_pred))
print(roc_auc_score(y_test_label, y_pred))

Accuracy: 0.7215836526181354

Classification Report:
               precision    recall  f1-score   support

         0.0       0.74      0.84      0.79       477
         1.0       0.69      0.53      0.60       306

    accuracy                           0.72       783
   macro avg       0.71      0.69      0.69       783
weighted avg       0.72      0.72      0.71       783

0.6871377481810333
